In [35]:
import pandas as pd
#Part 1 Implementation
# File paths
data_path = '/Users/anwayatkekar/Downloads/assignment2 (1)/adult.data'
test_path = '/Users/anwayatkekar/Downloads/assignment2 (1)/adult.test'


# Column names based on the adult.names files
column = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race",
    "sex", "capital_gain", "capital_loss", "hours_per_week",
    "native_country", "income"
]

# Loading the datasets
adultdata = pd.read_csv(data_path, names=column, na_values="?", skipinitialspace=True)
adulttest = pd.read_csv(test_path, names=column, na_values="?", skipinitialspace=True, skiprows=1)

print("Summary statistics for the adult data:")
print(adultdata.describe())

print("\nSummary statistics for the adult test")
print(adulttest.describe())


Summary statistics for the adult data:
                age        fnlwgt  education_num  capital_gain  capital_loss  \
count  32561.000000  3.256100e+04   32561.000000  32561.000000  32561.000000   
mean      38.581647  1.897784e+05      10.080679   1077.648844     87.303830   
std       13.640433  1.055500e+05       2.572720   7385.292085    402.960219   
min       17.000000  1.228500e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.178270e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.783560e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.370510e+05      12.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours_per_week  
count    32561.000000  
mean        40.437456  
std         12.347429  
min          1.000000  
25%         40.000000  
50%         40.000000  
75%         45.000000  
max         99.000000  

Summary statistics for 

In [36]:
# Standardizing the income column in the test dataset
adulttest['income'] = adulttest['income'].str.replace('.', '', regex=False)

# Checking for missing values in both datasets
missingdata = adultdata.isnull().sum()
missingtest = adulttest.isnull().sum()

# Outputting the summary of missing values for both datasets
print("Column names Summary:")
print("Adult.data:")
for column, value in missingdata.items():
    print(f"{column}: {value}")

print("\nAdult.test:")
for column, value in missingtest.items():
    print(f"{column}: {value}")



Column names Summary:
Adult.data:
age: 0
workclass: 1836
fnlwgt: 0
education: 0
education_num: 0
marital_status: 0
occupation: 1843
relationship: 0
race: 0
sex: 0
capital_gain: 0
capital_loss: 0
hours_per_week: 0
native_country: 583
income: 0

Adult.test:
age: 0
workclass: 963
fnlwgt: 0
education: 0
education_num: 0
marital_status: 0
occupation: 966
relationship: 0
race: 0
sex: 0
capital_gain: 0
capital_loss: 0
hours_per_week: 0
native_country: 274
income: 0


In [37]:
# Imputing missing values with the most frequent category for each column with missing values
for column in ['workclass', 'occupation', 'native_country']:
    data_m = adultdata[column].mode()[0]
    test_m = adulttest[column].mode()[0]
    
    adultdata.loc[adultdata[column].isnull(), column] = data_m
    adulttest.loc[adulttest[column].isnull(), column] = test_m

# Rechecking missing values after imputation
missingdata_after = adultdata.isnull().sum()
missingtest_after = adulttest.isnull().sum()

# Outputting the summary of missing values after imputation
print("Missing Values Summary After Imputation:")
print("adult.data:")
for column, value in missingdata_after.items():
    print(f"{column}: {value}")

print("\nadult.test:")
for column, value in missingtest_after.items():
    print(f"{column}: {value}")


Missing Values Summary After Imputation:
adult.data:
age: 0
workclass: 0
fnlwgt: 0
education: 0
education_num: 0
marital_status: 0
occupation: 0
relationship: 0
race: 0
sex: 0
capital_gain: 0
capital_loss: 0
hours_per_week: 0
native_country: 0
income: 0

adult.test:
age: 0
workclass: 0
fnlwgt: 0
education: 0
education_num: 0
marital_status: 0
occupation: 0
relationship: 0
race: 0
sex: 0
capital_gain: 0
capital_loss: 0
hours_per_week: 0
native_country: 0
income: 0


In [38]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

#Category columns
cols_c = [col for col in adultdata.columns if adultdata[col].dtype == "object" and col != "income"]

e1= OneHotEncoder(sparse_output=False, drop='first')

#Concating adultdata set and adulttest set
cols_category = pd.concat([adultdata[cols_c], adulttest[cols_c]])

e1.fit(cols_category)

encoded_d = e1.transform(adultdata[cols_c])
encoded_t = e1.transform(adulttest[cols_c])

data_e = pd.DataFrame(encoded_d, columns=e1.get_feature_names_out(cols_c))
test_e = pd.DataFrame(encoded_t, columns=e1.get_feature_names_out(cols_c))

#Adding num cols
num_cols = adultdata.select_dtypes(include=['int64', 'float64']).columns

data_e[num_cols] = adultdata[num_cols]
test_e[num_cols] = adulttest[num_cols]

data_e['income'] = adultdata['income'].map({'<=50K': 0, '>50K': 1})
test_e['income'] = adulttest['income'].map({'<=50K': 0, '>50K': 1})

In [39]:
#Part 2 Implementation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data_a = data_e.drop(columns=['income'])
data_b = data_e['income']
test_a = test_e.drop(columns=['income'])
test_b = test_e['income']

# Initialize logistic regression model with 'liblinear' solver
logistic_model = LogisticRegression(solver='liblinear')

# Training the model
logistic_model.fit(data_a, data_b)

# Predicting on the test data
test_pred = logistic_model.predict(test_a)

# Calculate classification error rate
classification_error_rate = 1 - accuracy_score(test_b, test_pred)
accuracy = accuracy_score(test_b ,test_pred)

print("Error Rate:", classification_error_rate*100)
print("Accuracy:", accuracy*100)

Error Rate: 20.035624347398805
Accuracy: 79.96437565260119


In [40]:
#Part 3 Implementation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Feature Engineering technique
scaler = StandardScaler()
data_a_scaled = scaler.fit_transform(data_a)
test_a_scaled = scaler.transform(test_a)

# Initializing logistic regression model
logistic_model = LogisticRegression(solver='liblinear', max_iter=1000)

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l1', 'l2']  # Penalty term ('l1': Lasso, 'l2': Ridge)
}

# Performing grid search with cross-validation
grid_search = GridSearchCV(logistic_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(data_a_scaled, data_b)

# Get the best model from grid search
best_logistic_model = grid_search.best_estimator_

# Predicting on the test data
pred_logistic = best_logistic_model.predict(test_a_scaled)

# Calculating classification error rate
classification_error_rate_logistic = 1 - accuracy_score(test_b, pred_logistic)
accuracy = accuracy_score(test_b ,pred_logistic)


print("Modified Error Rate:", classification_error_rate_logistic*100)
print("Modified Accuracy ", accuracy*100 )

Modified Error Rate: 14.888520361157177
Modified Accuracy  85.11147963884282


In [41]:
#Part 4 implementation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

percentages = [50, 60, 70, 80, 90]

# Initialize lists to store error rates for each X
error_rates = {percentage: [] for percentage in percentages}

num_iterations = 5

for percentage in percentages:
    for _ in range(num_iterations):
        # Randomly sample the training dataset with the given percentage
        data_a_sampled, _, y_train_sampled, _ = train_test_split(data_a, data_b, train_size=percentage/100, stratify=data_b)
        
        # Train the classifier (logistic regression) using the down-sampled training dataset
        logistic_model = LogisticRegression(solver='liblinear', max_iter=1000)
        logistic_model.fit(data_a_sampled, y_train_sampled)
        
        # Evaluate the classifier using the evaluation dataset
        b_pred = logistic_model.predict(test_a)
        error_rate = 1 - accuracy_score(test_b, b_pred)
        
        # Record the error rate
        error_rates[percentage].append(error_rate)

# Calculate mean and deviation for each X
mean_error_rates = {percentage: np.mean(error_rates[percentage]) for percentage in percentages}
deviation_error_rates = {percentage: np.std(error_rates[percentage]) for percentage in percentages}

# Print the results
print("Error Rates Summary for Different Percentages:")
for percentage in percentages:
    print(f"X = {percentage}% - MEAN: {mean_error_rates[percentage]:.6f}, STD: {deviation_error_rates[percentage]:.6f}")



Error Rates Summary for Different Percentages:
X = 50% - MEAN: 0.200528, STD: 0.000218
X = 60% - MEAN: 0.200319, STD: 0.000189
X = 70% - MEAN: 0.200528, STD: 0.000361
X = 80% - MEAN: 0.200418, STD: 0.000455
X = 90% - MEAN: 0.200233, STD: 0.000198


In [42]:
#Part 5 implementation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'max_depth': [None, 5, 10, 15],  
    'min_samples_split': [2, 3, 5, 8],  
    'min_samples_leaf': [1, 2, 3, 4],  
    'max_features': ['sqrt', 'log2', None]  
}


decision_tree = DecisionTreeClassifier()

grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=7, scoring='f1')

grid_search.fit(data_a, data_b)

# Getting the best model
best_decision_tree = grid_search.best_estimator_

# Predictions for test set using the best model
predictions_y = best_decision_tree.predict(test_a)

accuracy_dt = accuracy_score(test_b, predictions_y)

error_rate_dt = 1 - accuracy_dt
print(f'Error rate: {error_rate_dt*100}')


# Accuracy score for the best model
print(f'Accuracy: {accuracy_dt*100}')



Error rate: 13.653952459922614
Accuracy: 86.3460475400774
